In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, GRU
from tensorflow.keras.callbacks import EarlyStopping




In [4]:
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""


tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


input_sequences = []
for line in text.split(". "):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

X, labels = input_sequences[:,:-1], input_sequences[:,-1]
y = to_categorical(labels, num_classes=total_words)


In [5]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(GRU(150, return_sequences=True))
model.add(GRU(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [6]:
early_stop = EarlyStopping(monitor='loss', patience=3)
epochs_list = [20, 60, 70]
history_dict = {}

for epochs in epochs_list:
    print(f"\nTraining model with {epochs} epochs...\n")
    history = model.fit(X, y, epochs=epochs, verbose=1, callbacks=[early_stop])
    history_dict[epochs] = history



Training model with 20 epochs...

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.0256 - loss: 3.7134  
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1186 - loss: 3.6936 
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1082 - loss: 3.6703 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1082 - loss: 3.6400
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0873 - loss: 3.6002 
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.1186 - loss: 3.5213 
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1082 - loss: 3.4473 
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1082 - loss: 3.4648
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.1082 - loss: 3.4302 
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.1338 - loss: 3.3725
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1546 - loss: 3.3166 
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [7]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text
seed_text = "Once upon a time"
print(generate_text(seed_text, 10, max_sequence_len))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Once upon a time there was a little girl named red riding hood hood


In [12]:
!pip install keras-tuner
from keras_tuner import RandomSearch
def build_model(hp):
    model = Sequential()
    model.add(Embedding(total_words, hp.Int('embedding_dim', 50, 150, step=50), input_length=max_sequence_len - 1))
    model.add(GRU(hp.Int('gru_units_1', 100, 300, step=50), return_sequences=True))
    model.add(GRU(hp.Int('gru_units_2', 50, 150, step=50)))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=5)
tuner.search(X, y, epochs=10, validation_split=0.2)
best_model = tuner.get_best_models(num_models=1)[0]


Trial 5 Complete [00h 00m 10s]
val_accuracy: 0.09090909361839294

Best val_accuracy So Far: 0.1818181872367859
Total elapsed time: 00h 00m 43s


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
